### 1. Crawling the information from website of stock news  http://www.aastocks.com/en/stocks/news/aafn-company-news as below and create news repository by extracting the information from the html content

In [ ]:
import re
import requests
from lxml import etree
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin
import json

import pandas as pd
#pd.set_option('display.max_columns', None)
import numpy as np
import numexpr as ne

#!pip install selenium
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

#webdriver.Chrome().browser. find_elements(By.CLASS_NAME,'inline_block') all '%inline_block%' bars <div class="inline_block lastupdsep"><div>/ (By.CSS_SELECTOR,'p.inline_block')all exact bars <p class="inline_block">/ (By.PARTIAL_LINK_TEXT,'')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
#browser.get("http://music.163.com")
#print(browser.page_source)
# browser.get(url)
# time.sleep(3)

#browser.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


**Headline | Releasing time | Polarity**

In [ ]:
url = "http://www.aastocks.com/en/stocks/news/aafn-company-news"
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    "Referer": "http://www.aastocks.com/en/stocks/news/aafn/latest-news/0"
}
# How many times scrolls to bottum is required in initial url?
times = 1

#def scrape(url, times): #def scrapeWeeklyCompNews(url, times):

if not url.startswith('http'):
    raise Exception('URLs need to start with "http"')

browser = webdriver.Chrome('chromedriver',options=chrome_options)
print('Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)')
browser.get(url)
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 5 # load all weekly news (15 pages) # You can set your own pause time. My laptop is a bit slow so I use 1 sec
#screen_height = browser.execute_script("return window.screen.height;")   # get the screen height of the web
previous_height = browser.execute_script("return document.body.scrollHeight;")  # starting height
counter = 1

while True: # load all info in initial page
    # scroll one screen height each time
    browser.execute_script("window.scrollTo(0, {previous_height}*{i});".format(previous_height=previous_height, i=counter))
    #html = browser.find_element_by_tag_name('html') # Or direct send key
    #html.send_keys(Keys.END)
    time.sleep(scroll_pause_time) # load all weekly news (15 pages)

    # whens.extend(t.text for t in list(browser.find_elements(By.CSS_SELECTOR,'div.inline_block')) if len(t.text)==16)
    # print(len(whens),'whens inserted: [..,"'+whens[-1]+'"],',counter,'pages done')
    counter += 1

    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    # scroll_height = browser.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if counter > times:  #if (screen_height) * i > scroll_height:
      break

## get ['Releasing time'], ['Polarity']
# after loading all documents..
whens = [tve.text for i,tve in enumerate(browser.find_elements(By.CSS_SELECTOR,'div.inline_block')) if len(tve.text)==16]
print(len(whens),'whens inserted: [..,"'+whens[-1]+'"],',counter-1,'pages done')
pos = [] # pos = [int(tve.text.replace('Positive','')) for i,tve in enumerate(browser.find_elements(By.CLASS_NAME,'divBullish'))]# if i%2 == 0]# even i for Bullish ,odd i for Bearish
neg = [] # neg = [int(tve.text.replace('Negative','')) for i,tve in enumerate(browser.find_elements(By.CLASS_NAME,'divBearish'))]# if i%2 != 0]# even i for Bullish ,odd i for Bearish
#VE = list(set(pos)-set(neg))  # VE = eval(tve.find_all(class_='value')[1].text+'-'+tve.find_all(class_='value')[2].text)
Polarity = []
for tve in list(browser.find_elements(By.CLASS_NAME,'div_VoteTotal')): # even i divBullish ,odd i for Bearish

  Bullish = int(tve.text.split('tive')[1].replace('Nega',''))
  Bearish = int(tve.text.split('tive')[2])
  VE = Bullish-Bearish

  pos.append(Bullish)
  neg.append(Bearish)
  Polarity.append('positive' if VE > 0 else 'negative' if VE < 0 else 'neutral')
print(len(Polarity),'Polarity inserted: [..,"'+Polarity[-1]+'"],',counter-1,'pages done')

## get ['Headline'] & news follow links
headlines = [] #headlines = [h.text for h in list(browser.find_elements(By.CLASS_NAME,'newshead4'))]
links = []
for h in list(browser.find_elements(By.XPATH,"//div[contains(@class, 'newshead4')]/a")):
  headlines.append(h.text)
  links.append(h.get_attribute('href'))
print(len(headlines),'headlines inserted: [..,"'+headlines[-1]+'"],',counter-1,'pages done')
print(len(links),'links inserted: [..,"'+links[-1]+'"],',counter-1,'pages done')
print('Thank you for your patience :)')

# if __name__ == '__main__':
#     scrape(url, 1) # How many times scrolls to bottum is required in initial url?

Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)
300 links inserted: [..,"http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1189660/company-news/HK6"], 1 pages done
Thank you for your patience :)


In [ ]:
browser.close() # close initial page

In [ ]:
links # check validity for next scrape

['http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191473/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191470/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191461/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191454/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191456/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191431/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191420/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191423/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191414/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191408/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191407/company-news/HK6',
 'http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191399/company-news/HK6',
 'http://www.aastocks.com/en

In [ ]:
# concat all columns
df = pd.DataFrame({'Headline':headlines, 'Releasing time':whens, 'Polarity':Polarity})
df = df.convert_dtypes()
df

,Headline,Releasing time,Polarity
0,PBOC Reportedly Accepts Ant's Application to F...,2022/06/17 19:05,positive
1,HKD336.4M Southbound Trading Net Inflow to GCL...,2022/06/17 18:25,positive
2,CKA: Harbour Plaza Resort City Hotel Rooms Con...,2022/06/17 17:39,positive
3,PARKnSHOP: 1Q Turnover Posts Double-digit Grow...,2022/06/17 17:14,positive
4,"ByteDance Disbands 101 Studio, Sacks About 150...",2022/06/17 17:12,positive
...,...,...,...
295,<Resumption Ann>CHINA JINMAO Proposes Privatiz...,2022/06/10 08:20,positive
296,Alibaba’s Shr Price Plunges over 8%; Ant: Focu...,2022/06/10 08:15,positive
297,<US Stocks>DJIA Craters 638 pts or 1.9%; Aliba...,2022/06/10 08:12,positive
298,BILIBILI-SW: Covid Not to Hurt Long-term Funda...,2022/06/10 07:30,negative


In [ ]:
df['Headline']

0      PBOC Reportedly Accepts Ant's Application to F...
1      HKD336.4M Southbound Trading Net Inflow to GCL...
2      CKA: Harbour Plaza Resort City Hotel Rooms Con...
3      PARKnSHOP: 1Q Turnover Posts Double-digit Grow...
4      ByteDance Disbands 101 Studio, Sacks About 150...
                             ...                        
295    <Resumption Ann>CHINA JINMAO Proposes Privatiz...
296    Alibaba’s Shr Price Plunges over 8%; Ant: Focu...
297    <US Stocks>DJIA Craters 638 pts or 1.9%; Aliba...
298    BILIBILI-SW: Covid Not to Hurt Long-term Funda...
299    CBIRC: Large State-owned Banks Required to Ens...
Name: Headline, Length: 300, dtype: string

**Company name | Stock Code | Abstract** (loading and retreive requiering ~10+time.sleep(2) second per news, using links extracted from above)

In [ ]:
fol_headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    "Referer": "http://www.aastocks.com/en/stocks/news/aafn-company-news"
    }
symbols = []
names = []
abstracts = []
browser = webdriver.Chrome('chromedriver',options=chrome_options)
print('Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)')
# 300 links requiring ~1hr (including 10 mins time.sleep), ~12 sec/link to retrieve [name], [symbol] & [abstract]
for link in links: # follow each news links to get [name], [symbol] & [abstract]
  #wait = WebDriverWait(browser, scroll_pause_time) # we don't want scroll initial page again https://stackoverflow.com/questions/70683534/python-selenium-iterate-table-of-links-clicking-each-link
  print(link)
  browser.get(link)
  print('Loading 2 sec to pretend as a human..')
  time.sleep(2)
  abstracts.append(browser.find_element(By.XPATH,'//p').text)
  print(len(abstracts),'abstracts inserted')
  #sym = browser.execute_script('return NewsRelativeSymbol') # str[:5]
  # sess = requests.session()
  # response = sess.get(link, headers=fol_headers)
  # soup = BeautifulSoup(response.json(),'lxml') #.content html.parser
  #abstract.append(soup.find("p").text) #.get_text(strip=True,separator='<br>').split('<br>')#.text.strip().split('\n')[0]#.strip()#.contents.strip()#class_= 'newscontent5 fLevel3')
  #sym = soup.find("a",class_='jsStock jsBmpStock').get('sym')
  try:
  #   sym = soup.find("a",class_='jsStock jsBmpStock')['sym']
    button = browser.find_element(By.XPATH,'//a[@class="jsStock jsBmpStock"]')#.get_attribute('href') # 1st relevant symbol appears

  except Exception as e:
    print(e,'\nNo Ticker button appears, getting Related Symbol')
    # json_string = re.search(r'NewsRelativeSymbol\s*=\s*(.*)', str(soup.find('script')), flags=re.DOTALL)
    # #sym = soup.find('a',class_='more-lnk')
    # sym = json.loads(json_string[1])
    # above return empty...
    button = browser.find_element(By.LINK_TEXT,"Related News") # related symbol from related news link

  browser.execute_script("arguments[0].click();", button)
  symbols.append(browser.find_element(By.XPATH,'//div[@id="SQ_Symbol"]').text[1:6])
  print(len(symbols),'symbols inserted: [..,"'+symbols[-1]+'"],')
  names.append(browser.find_element(By.XPATH,'//label[@id="SQ_Name"]').text)
  print(len(names),'company names inserted: [..,"'+names[-1]+'"]')


print('Thank you for your patience :)')

Hi, I will be digging so hard for a while, Please take a break or have a cup of tea ;)
http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191473/company-news/HK6
Loading 2 sec to pretend as a human..
1 abstracts inserted
1 symbols inserted: [..,"09988"],
1  company names inserted: [..,"BABA-SW"]
http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191470/company-news/HK6
Loading 2 sec to pretend as a human..
2 abstracts inserted
2 symbols inserted: [..,"03800"],
2  company names inserted: [..,"GCL TECH"]
http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191461/company-news/HK6
Loading 2 sec to pretend as a human..
3 abstracts inserted
3 symbols inserted: [..,"01113"],
3  company names inserted: [..,"CK ASSET"]
http://www.aastocks.com/en/stocks/news/aafn-con/NOW.1191454/company-news/HK6
Loading 2 sec to pretend as a human..
4 abstracts inserted
4 symbols inserted: [..,"00001"],
4  company names inserted: [..,"CKH HOLDINGS"]
http://www.aastocks.com/en/stocks/news/aafn-con/NOW.119145

In [ ]:
browser.close()

In [ ]:
# after finish digging..
df['Company name'] = names
df['Stock Code'] = symbols
df['Abstract'] = abstracts

**Headline | Releasing time | Company name | Stock Code | Abstract | Polarity**

In [ ]:
df = df[['Headline', 'Releasing time', 'Company name', 'Stock Code', 'Abstract','Polarity']]
df = df.convert_dtypes()
df['Releasing time'] = pd.to_datetime(df['Releasing time'])
df['Stock Code'] = df['Stock Code'].astype('category')
df['Polarity'] = df['Polarity'].astype('category')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Headline        300 non-null    string        
 1   Releasing time  300 non-null    datetime64[ns]
 2   Company name    300 non-null    string        
 3   Stock Code      300 non-null    category      
 4   Abstract        300 non-null    string        
 5   Polarity        300 non-null    category      
dtypes: category(2), datetime64[ns](1), string(3)
memory usage: 15.2 KB


,Headline,Releasing time,Company name,Stock Code,Abstract,Polarity
0,PBOC Reportedly Accepts Ant's Application to F...,2022-06-17 19:05:00,BABA-SW,09988,The People's Bank of China (PBOC) has accepted...,positive
1,HKD336.4M Southbound Trading Net Inflow to GCL...,2022-06-17 18:25:00,GCL TECH,03800,"There was HKD336.4 million, HKD297.1 million a...",positive
2,CKA: Harbour Plaza Resort City Hotel Rooms Con...,2022-06-17 17:39:00,CK ASSET,01113,CK ASSET (01113.HK) -0.500 (-0.967%) Short...,positive
3,PARKnSHOP: 1Q Turnover Posts Double-digit Grow...,2022-06-17 17:14:00,CKH HOLDINGS,00001,"Norman Yum, Managing Director of PARKnSHOP Sup...",positive
4,"ByteDance Disbands 101 Studio, Sacks About 150...",2022-06-17 17:12:00,TENCENT,00700,"ByteDance disbanded its ""101 Studio"" gaming op...",positive
...,...,...,...,...,...,...
295,<Resumption Ann>CHINA JINMAO Proposes Privatiz...,2022-06-10 08:20:00,CHINA VAST,06166,CHINA VAST (06166.HK) 0.000 (0.000%) receiv...,positive
296,Alibaba’s Shr Price Plunges over 8%; Ant: Focu...,2022-06-10 08:15:00,BABA-SW,09988,Alibaba Group Holding (BABA.US) ’s share pr...,positive
297,<US Stocks>DJIA Craters 638 pts or 1.9%; Aliba...,2022-06-10 08:12:00,BABA-SW,09988,U.S. stocks declined as investors turned incre...,positive
298,BILIBILI-SW: Covid Not to Hurt Long-term Funda...,2022-06-10 07:30:00,BILIBILI-SW,09626,The first-quarter results of BILIBILI-SW (0962...,negative


In [ ]:
df.to_excel('/content/drive/MyDrive/ML_skillset/AAstocksWEEKLYcompNews.xlsx', index=True, header = df.columns) # excel can fixed grid size